<h1>Wyszukiwanie w tekstach</h1>

Kluczowym problemem nlp jest wyszukiwanie podobnych dokumentów.

ZADANIE: Prosze wczytać słownik i korpus.

In [2]:
from gensim import corpora, models, similarities
dictionary = corpora.Dictionary.load('tmp/deerwester.dict')
corpus = corpora.MmCorpus('tmp/deerwester.mm') # comes from the first tutorial, "From strings to vectors"
print(corpus)

C:\Users\user\Anaconda3\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


MmCorpus(9 documents, 12 features, 28 non-zero entries)


Bierzemy nowy dokument. 

In [3]:
doc = "Human computer interaction"
vec_bow = dictionary.doc2bow(doc.lower().split())
vec_bow

[(0, 1), (1, 1)]

W gensim najpierw inicjalizujemy query (tworzymy macierz podobieństw) a potem możemy querować podobne dokuenty. Do querowania używane jest podobieństwo cosinusowe.

In [5]:
index = similarities.MatrixSimilarity(corpus)

print(list(index))



sims = index[vec_bow] # perform a similarity query against the corpus

print(list(enumerate(sims))) # print (document_number, document_similarity) 2-tuples

[array([ 0.99999994,  0.23570226,  0.28867513,  0.23570226,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ], dtype=float32), array([ 0.23570226,  1.        ,  0.40824831,  0.33333334,  0.70710677,
        0.        ,  0.        ,  0.        ,  0.23570226], dtype=float32), array([ 0.28867513,  0.40824831,  1.        ,  0.61237246,  0.28867513,
        0.        ,  0.        ,  0.        ,  0.        ], dtype=float32), array([ 0.23570226,  0.33333334,  0.61237246,  1.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ], dtype=float32), array([ 0.        ,  0.70710677,  0.28867513,  0.        ,  0.99999994,
        0.        ,  0.        ,  0.        ,  0.        ], dtype=float32), array([ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        1.        ,  0.70710677,  0.57735026,  0.        ], dtype=float32), array([ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.70710677,  0.99999994,  0.816

ZADANIE: Proszę posortować korpus od najbardziej pasujących.

In [12]:
sims = sorted(enumerate(sims), key=lambda item: -item[1])
print(sims) # print sorted (document number, similarity score) 2-tuples

[(0, 0.81649655), (1, 0.28867513), (3, 0.28867513), (2, 0.0), (4, 0.0), (5, 0.0), (6, 0.0), (7, 0.0), (8, 0.0)]


ZADANIE: Proszę zaanalizować wynik. Jakie dokumenty okazały się najbardziej podobne.

ZADANIE: Proszę użyć tfidf do analogicznego zadania. Należy stworzyć odpowiednie funkcje które pozwolą nam yżywać róóżnych transformacji do querowania.

Czy wynik się różni? Co się zmieniło?

In [13]:
tfidf = models.TfidfModel(corpus, id2word=dictionary)
corpus_tfidf = tfidf[corpus]

vec_tfidf = tfidf[vec_bow]

index = similarities.MatrixSimilarity(corpus_tfidf)

sims = index[vec_tfidf]

print(list(enumerate(sims)))

#sims = sorted(enumerate(sims), key=lambda item: -item[1])
print(sims)

[(0, 0.81649655), (1, 0.31412902), (2, 0.0), (3, 0.34777319), (4, 0.0), (5, 0.0), (6, 0.0), (7, 0.0), (8, 0.0)]
[(0, 0.81649655), (3, 0.34777319), (1, 0.31412902), (2, 0.0), (4, 0.0), (5, 0.0), (6, 0.0), (7, 0.0), (8, 0.0)]


ZADANIE: Teraz proszę zrobić to samo z LSI

In [14]:
lsi = models.LsiModel(corpus, id2word=dictionary, num_topics=2)
vec_lsi = lsi[vec_bow] # convert the query to LSI space

index = similarities.MatrixSimilarity(lsi[corpus])

sims = index[vec_lsi] # perform a similarity query against the corpus
print(list(enumerate(sims)))

sims = sorted(enumerate(sims), key=lambda item: -item[1])
print(sims)

[(0, 0.99809301), (1, 0.93748635), (2, 0.99844527), (3, 0.9865886), (4, 0.90755945), (5, -0.12416792), (6, -0.10639259), (7, -0.098794639), (8, 0.050041765)]
[(2, 0.99844527), (0, 0.99809301), (3, 0.9865886), (1, 0.93748635), (4, 0.90755945), (8, 0.050041765), (7, -0.098794639), (6, -0.10639259), (5, -0.12416792)]


Proszę zauważyć że w LSI pasujące dokumenty nie muszą zawierać słów które występuja w query. Dlaczego?

Na początek niech wezmą tutorial gensim, pobiorą dokument i stworzą reprezentację. Potem będziemy opowiadać o LDA

ZADANIA:

1. Mamy posty (tweety) użytkowników. Bierzemy jakiegoś użytkownika (zakładamy, że znamy dobrze jego zachowanie) i pytamy się, którzy użytkownicy są do niego podobni. Jeśli ich znajdziemy będziemy mogli im dedykować analogiczne reklamy i treści.

Rozw: można potraktować posty tego użytkownika jak query i wyszukać najbardziej podobne dokumenty innych użytkowników. Jeśli uzytkownicy mają dużo postów to trzeba jakąś miarę na zbiorach postów - dla prostoty porównujemy średnie, ale można też jakieś linkage functions z hierarchicznego wziąć.


2. Pytamy się o czym jest dany tekst. 

Rozw: Najprostsze rozwiązanie to znalezienie najczęstszych słów. Bardzej zaawansowane polega na wyszukaniu topiców (LSA albo LDA) - wtedy mamy niezależne topici.

3. Mamy podział na topici. Bierzemy użytkownika. Pytamy się którzy użytkownicy są do niego podobni biorąc pod uwagę tylko określony topic (na przykład pytamy sie którzy są podobni w kategorii sport. Zadanie możemy rozszerzyć na pytanie, którzy uyżytkownicy są do niego podobni biorąc pod uwagę kilka topiców na raz.

Rozw: Jeśli mamy topic albo topici to wystarczy zrzutować naszego użytkownika na tą podprzestrzeń. Tak samo robimy z innymi użytkownikami i szukamy najbliższych w tej podprzestrzeni.

4. Mamy podział na topici - które słowa charakteryzują najbardzie dany topic

Bierzemy te współrzędne (słowa) z największą normą z kolejnych wektorów własnych.

5. Jakie zdania najbardziej charakteryzują topic?

Jest to chyba opisane w https://towardsdatascience.com/nlp-for-topic-modeling-summarization-of-legal-documents-8c89393b1534 ale nie dokonca rozumiem. Jest tez notebook: https://github.com/chibueze07/Machine-Learning-In-Law/tree/master

6. Jakich osób (podmiotów) dotyczą topici. Trzeba zrobic POS i dla każdego podmiotu wyszukać podobieństwo do danego topicu. 


